In [1]:
import numpy as np
import pandas as pd
import math
from apriori import Apriori
from time import time
from helper import GetItemsetFromFile, GetSubsectionOfData, SplitDataIntoChunks, PrintTimeInfo, CreatePairDataFrame

In [2]:
# Get the retail data set
s_t = time()
retail_data = GetItemsetFromFile("retail.data")
f_t = time()
print("Time taken to read retail data: %.2f seconds" % (f_t - s_t))

Time taken to read retail data: 0.22 seconds


In [3]:
support = 0.05 # 5%

In [4]:
data = 0.2 # 20%
retail_length = len(retail_data)
data_length = math.floor(retail_length * data)
partial_data = []
did_not_divide_nicely = False
num_iterations = int(100 / (data * 100))
list_of_frequent_item_lists = []
partials = []

# Remainder after dividing by the fraction
if retail_length % (data * 100) != 0:
    did_not_divide_nicely = True
    # Left overs is the remainder after dividing
    # In the case of 20%, if you round you get 5 data lengths of 17632
    # 17632 * 5 = 88160
    # Actual length of data is 88162
    # This gets the remaining 2 rows we would have missed so we can cover the entire dataset
    leftovers = round(retail_length - (math.floor(retail_length / num_iterations) * num_iterations))
    
for i in range(1, num_iterations+1):
    data_size = data_length * i
    # if there was a remainder from division
    if did_not_divide_nicely == True:
        # If it is the last run of the loop
        if i == num_iterations:
            # Add the leftover value to the last pass to get the entire dataset
            data_size += leftovers

    partial_data = GetSubsectionOfData(retail_data[data_length * (i-1):data_size], 1)
    partials.append(partial_data)

    occ, freq = Apriori(partial_data, support / num_iterations, 2)
    list_of_frequent_item_lists.append(freq)


In [5]:
# Run Apriori at 5% support on 100% of data
occ, freq = Apriori(retail_data, support, 2)

In [6]:
false_positives = []
for frequency_list in list_of_frequent_item_lists:
    for item in frequency_list:
        if item not in freq:
            false_positives.append(item)